In [1]:
!pip install bitsandbytes transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 11.7 MB/s eta 0:00:0000:0100:01


In [2]:
from typing import Union, List
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from IPython.display import HTML


class BinocularModel:
    def __init__(self, observer_name: str, performer_name: str, 
                 device_1: str = "cuda:0", device_2: str = "cuda:1"):
        self.DEVICE_1 = device_1
        self.DEVICE_2 = device_2

        torch.set_grad_enabled(False)

        self.tokenizer = AutoTokenizer.from_pretrained(observer_name)
        assert (self.tokenizer.vocab == AutoTokenizer.from_pretrained(performer_name).vocab)

        self.observer_model = AutoModelForCausalLM.from_pretrained(observer_name,
                                                                   device_map={"": self.DEVICE_1},
                                                                   trust_remote_code=True,
                                                                   torch_dtype=torch.bfloat16)
        self.observer_model.eval()

        self.performer_model = AutoModelForCausalLM.from_pretrained(performer_name,
                                                                    device_map={"": self.DEVICE_2},
                                                                    trust_remote_code=True,
                                                                    torch_dtype=torch.bfloat16)
        self.performer_model.eval()

        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        self.loss_fn = torch.nn.CrossEntropyLoss(reduction='none')
        self.softmax_fn = torch.nn.Softmax(dim=-1)

    def tokenize(self, batch):
        encodings = self.tokenizer(batch, return_tensors="pt", 
        padding="longest" if len(batch) > 1 else False, truncation=True,
        max_length=512, return_token_type_ids=False).to(self.DEVICE_1)
        return encodings

    @torch.inference_mode()
    def get_logits(self, encodings):
        observer_logits = self.observer_model(**encodings.to(self.DEVICE_1)).logits
        performer_logits = self.performer_model(**encodings.to(self.DEVICE_2)).logits
        torch.cuda.synchronize()

        return observer_logits, performer_logits

    def perplexity(self, encoding, logits):
        shifted_logits = logits[..., :-1, :].contiguous()
        shifted_labels = encoding.input_ids[..., 1:].contiguous().to("cpu")
        shifted_attention_mask = encoding.attention_mask[..., 1:].contiguous().to("cpu")

        ppl = self.loss_fn(shifted_logits.transpose(1, 2).to("cpu"), shifted_labels) * shifted_attention_mask
        ppl = ppl.sum(1) / shifted_attention_mask.sum(1)

        return ppl.to("cpu").float().numpy()

    def cross_perplexity(self, observer_logits, performer_logits, encoding):
        V = observer_logits.shape[-1]
        S = observer_logits.shape[-2]

        performer_probs = self.softmax_fn(performer_logits).view(-1, V).to("cpu")
        observer_scores = observer_logits.view(-1, V).to("cpu")

        xppl = self.loss_fn(observer_scores, performer_probs).view(-1, S)
        padding_mask = (encoding.input_ids != self.tokenizer.pad_token_id).type(torch.uint8).to("cpu")

        xppl = (xppl * padding_mask).sum(1) / padding_mask.sum(1)

        return xppl.to("cpu").float().numpy()

    def binocular_score(self, text):
        batch = [text] if isinstance(text, str) else text
        encodings = self.tokenize(batch)
        observer_logits, performer_logits = self.get_logits(encodings)
        ppl = self.perplexity(encodings, observer_logits)
        xppl = self.cross_perplexity(observer_logits, performer_logits, encodings)

        return (ppl / xppl).tolist()

    def generate_html(self, tokens, scores, higher_surprise_darker_color: bool=True):
        html = "<p><pre>" + tokens[0]
        for token, score in zip(tokens[1:], scores[0]):
            if higher_surprise_darker_color:
                color_value = 255 * score * 0.7
                html += f"<span style='background-color: rgb(255, {255-color_value}, {255-color_value}); color: black;'>{token}</span>"
            else:
                color_value = 255 - (255 * (1 - score) * 0.7)
                html += f"<span style='background-color: rgb(255, {color_value}, {color_value}); color: black;'>{token}</span>"
        html += "</pre></p>"
        return html

    def display_html_markup(self, text, higher_surprise_darker_color: bool=True):
        encoding = self.tokenize([text])
        observer_logits, performer_logits = self.get_logits(encoding)

        S = observer_logits.shape[-2]
        V = observer_logits.shape[-1]

        shifted_logits = observer_logits[..., :-1, :].contiguous().to('cpu')
        shifted_labels = encoding.input_ids[..., 1:].contiguous().to('cpu')

        ppl = self.loss_fn(shifted_logits.transpose(1, 2), shifted_labels).float()

        performer_probs = self.softmax_fn(performer_logits).view(-1, V).to("cpu")
        observer_scores = observer_logits.view(-1, V).to("cpu")

        xppl = self.loss_fn(observer_scores[:-1], performer_probs[:-1]).view(-1, S - 1).to("cpu").float()

        tokens = [self.tokenizer.decode([tok], clean_up_tokenization_spaces=False) for tok in encoding.input_ids.squeeze().tolist()]
        
        score = (ppl / torch.max(ppl)) / (xppl / torch.max(xppl))
        normalized_score = score / torch.max(score)
        html_output = self.generate_html(tokens, normalized_score, higher_surprise_darker_color)

        return html_output

In [3]:
observer_name = "tiiuae/falcon-7b-instruct"
performer_name = "tiiuae/falcon-7b"

binocular_model = BinocularModel(observer_name, performer_name)

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [4]:
# human
text = """
void VPlanTransforms::createAndOptimizeReplicateRegions(VPlan &Plan) {
  // Convert masked VPReplicateRecipes to if-then region blocks.
  addReplicateRegions(Plan);

  bool ShouldSimplify = true;
  while (ShouldSimplify) {
    ShouldSimplify = sinkScalarOperands(Plan);
    ShouldSimplify |= mergeReplicateRegionsIntoSuccessors(Plan);
    ShouldSimplify |= VPlanTransforms::mergeBlocksIntoPredecessors(Plan);
  }
}
"""
score = binocular_model.binocular_score(text)[0]
print('Most likely AI-generated' if score < 0.85 else 'Most likely human-generated')
print('Score:', score)
display(HTML(binocular_model.display_html_markup(text, higher_surprise_darker_color=True)))

Most likely human-generated
Score: 1.1265822649002075


2024-07-30 13:39:45.353981: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 13:39:45.355205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 13:39:45.543768: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# llm
text = """
Matcher* Matcher::unlinkNode(Matcher* Other) {
    if (Other == this) {
        Matcher* next = this->next;
        this->next = nullptr;
        return next;
    }
    else {
        Matcher* current = this;
        while (current->next!= Other) {
            current = current->next;
        }
        Matcher* next = current->next;
        current->next = nullptr;
        return this;
    }
}
"""
score = binocular_model.binocular_score(text)[0]
print('Most likely AI-generated' if score < 0.85 else 'Most likely human-generated')
print('Score:', score)
display(HTML(binocular_model.display_html_markup(text, higher_surprise_darker_color=True)))

Most likely AI-generated
Score: 0.75
